In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/contradictory-my-dear-watson/sample_submission.csv
/kaggle/input/contradictory-my-dear-watson/train.csv
/kaggle/input/contradictory-my-dear-watson/test.csv


In [2]:
df = pd.read_csv("/kaggle/input/contradictory-my-dear-watson/test.csv")

In [3]:
df['text_input'] = df['premise'] + '<sep>' + df['hypothesis']
df['text_input'] = df['text_input'].apply(lambda x:x.split("<sep>"))

In [4]:
df

,id,premise,hypothesis,lang_abv,language,text_input
0,c6d58c3f69,بکس، کیسی، راہیل، یسعیاہ، کیلی، کیلی، اور کولم...,"کیسی کے لئے کوئی یادگار نہیں ہوگا, کولمین ہائی...",ur,Urdu,[بکس، کیسی، راہیل، یسعیاہ، کیلی، کیلی، اور کول...
1,cefcc82292,هذا هو ما تم نصحنا به.,عندما يتم إخبارهم بما يجب عليهم فعله ، فشلت ال...,ar,Arabic,"[هذا هو ما تم نصحنا به., عندما يتم إخبارهم بما..."
2,e98005252c,et cela est en grande partie dû au fait que le...,Les mères se droguent.,fr,French,[et cela est en grande partie dû au fait que l...
3,58518c10ba,与城市及其他公民及社区组织代表就IMA的艺术发展进行对话&amp,IMA与其他组织合作，因为它们都依靠共享资金。,zh,Chinese,"[与城市及其他公民及社区组织代表就IMA的艺术发展进行对话&amp, IMA与其他组织合作，..."
4,c32b0d16df,Она все еще была там.,"Мы думали, что она ушла, однако, она осталась.",ru,Russian,"[Она все еще была там., Мы думали, что она ушл..."
...,...,...,...,...,...,...
5190,5f90dd59b0,نیند نے وعدہ کیا کہ موٹل نے سوال میں تحقیق کی.,نیمیتھ کو موٹل کی تفتیش کے لئے معاوضہ دیا جارہ...,ur,Urdu,[نیند نے وعدہ کیا کہ موٹل نے سوال میں تحقیق کی...
5191,f357a04e86,The rock has a soft texture and can be bough...,The rock is harder than most types of rock.,en,English,[The rock has a soft texture and can be boug...
5192,1f0ea92118,她目前的存在，并考虑到他与沃佛斯顿争执的本质，那是尴尬的。,她在与Wolverstone的打斗结束后才在场的事实被看作是很尴尬的。,zh,Chinese,"[她目前的存在，并考虑到他与沃佛斯顿争执的本质，那是尴尬的。, 她在与Wolverstone..."
5193,0407b48afb,isn't it i can remember i've only been here ei...,I could see downtown Dallas from where I lived...,en,English,[isn't it i can remember i've only been here e...


In [5]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=c9fa491ec0d1185dac7bb4cc7fe1142636ed65a588dffe2f7f21881ab6277cf2
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
Note: you may need to restart the kernel to use updated packages.


In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model_name = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [7]:
pairs = df['text_input'].tolist()

# Create empty data list
data = []
model.cuda()
# Iterate over premise and hypothesis pairs
for premise, hypothesis in pairs:
    # Tokenize and get predictions
    input = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt").to(device)
    output = model(input["input_ids"].to(device))
    prediction = torch.softmax(output["logits"][0], -1).tolist()
    label_names = [0, 1, 2]
    prediction = {name: round(pred * 100, 1) for pred, name in zip(prediction, label_names)}
    
    # Get the label with the highest probability
    highest_label = max(prediction, key=prediction.get)
    
    # Append the highest label to data list
    data.append({"Prediction": highest_label})

# Create Pandas dataframe from data list
df_predictions = pd.DataFrame(data)

In [8]:
sample= pd.read_csv('/kaggle/input/contradictory-my-dear-watson/sample_submission.csv')
sample['prediction'] = df_predictions['Prediction'].values
sample.to_csv("submission.csv", index=False)
sample

,id,prediction
0,c6d58c3f69,2
1,cefcc82292,1
2,e98005252c,0
3,58518c10ba,1
4,c32b0d16df,1
...,...,...
5190,5f90dd59b0,0
5191,f357a04e86,1
5192,1f0ea92118,0
5193,0407b48afb,0
